In [ ]:
%scala
val containerName = "dsdcontainer1"
val storageAccountName = "dsdsa1"
val sas = "sp=racwdli&st=2022-12-16T13:56:14Z&se=2023-01-01T21:56:14Z&spr=https&sv=2021-06-08&sr=c&sig=1VocpXfRdLv%2BOMF9L<sampletoken>"
val config = "fs.azure.sas." + containerName + "." + storageAccountName + ".blob.core.windows.net"

containerName: String = dsdcontainer1
storageAccountName: String = dsdsa1
sas: String = sp=racwdli&st=2022-12-16T13:56:14Z&se=2023-01-01T21:56:14Z&spr=https&sv=2021-06-08&sr=c&sig=1VocpXfRdLv%2BOMF9LiRRm2oCWxGEybmpsDPTR5jwqCE%3D
config: String = fs.azure.sas.dsdcontainer1.dsdsa1.blob.core.windows.net

In [ ]:
%scala
val containerName = "yelpdeltatables"
val storageAccountName = "dsdsa1"
val sas = "sp=racwdli&st=2022-12-16T14:47:39Z&se=2023-01-01T22:47:39Z&spr=https&sv=2021-06-08&sr=c&sig=tusozHSuVhzLCztnQuQ8SKuEYcR%2F<sampletoken>"
val config = "fs.azure.sas." + containerName + "." + storageAccountName + ".blob.core.windows.net"

containerName: String = yelpdeltatables
storageAccountName: String = dsdsa1
sas: String = sp=racwdli&st=2022-12-16T14:47:39Z&se=2023-01-01T22:47:39Z&spr=https&sv=2021-06-08&sr=c&sig=tusozHSuVhzLCztnQuQ8SKuEYcR%2F4EPy8XmquycaHh0%3D
config: String = fs.azure.sas.yelpdeltatables.dsdsa1.blob.core.windows.net

In [ ]:
# %scala
# dbutils.fs.mount(
#   source = "wasbs://dsdcontainer1@dsdsa1.blob.core.windows.net/",
#   mountPoint = "/mnt/yelpdata/",
#   extraConfigs = Map(config -> sas))

In [ ]:
# %scala
# dbutils.fs.mount(
#   source = "wasbs://yelpdeltatables@dsdsa1.blob.core.windows.net/",
#   mountPoint = "/mnt/yelpdelta/",
#   extraConfigs = Map(config -> sas))

In [ ]:
curated_data = spark.read.csv("/mnt/yelpdata/curated.csv",header=True,multiLine = True)

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS DSDDB

In [ ]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
dfq2 = curated_data.where((curated_data.categories.contains("Gas Stations")) & (curated_data.categories.contains('Food')) | (curated_data.categories.contains('Fast Food')) | (curated_data.categories.contains('Restaurants')) | (curated_data.categories.contains('Pizza')))

df_query2 = dfq2.filter((dfq2.average_stars > 3.5) & (dfq2.state=="FL"))

df_query2 = df_query2.select('business_name','average_stars','city','postal_code')

df_querysorted2 = df_query2.sort(df_query2.average_stars.desc())

df_querysorted2 = df_querysorted2.dropDuplicates()

df_querysorted2 = df_querysorted2.limit(100000)

df_querysorted2.head()

Out[120]: Row(business_name="Mazzaro's Italian Market", average_stars='5.0', city='Saint Petersburg', postal_code='33713')

In [ ]:
df_querysorted2.write\
              .partitionBy("city")\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase1/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.gasstations_and_food_availability_florida USING DELTA LOCATION '/mnt/yelpdelta/usecase1/'")

Out[122]: DataFrame[]

In [ ]:
df3 = curated_data.filter(curated_data.elite != '')
df3.withColumn("fans",F.col('fans').cast('int'))
cols = ["compliment_hot", "compliment_more", "compliment_profile","compliment_cute","compliment_list","compliment_note","compliment_plain","compliment_cool","compliment_funny","compliment_writer","compliment_photos"]
for col_name in cols:
    df3 = df3.withColumn(col_name, col(col_name).cast('int'))
    
df3.withColumn("review_count",F.col('review_count').cast('int'))

def complimentsum(a,b,c,d,e,f,g,h,i,j,k):
    col_sum = a+b+c+d+e+f+g+h+i+j+k
    return col_sum

df3 = df3.na.drop(subset=["compliment_hot", "compliment_more", "compliment_profile","compliment_cute","compliment_list","compliment_note","compliment_plain","compliment_cool","compliment_funny","compliment_writer","compliment_photos"])

new_f = F.udf(complimentsum, IntegerType())

df_usercomp = df3.withColumn("total_compliments",
                          new_f("compliment_hot", "compliment_more", "compliment_profile","compliment_cute","compliment_list","compliment_note","compliment_plain","compliment_cool","compliment_funny","compliment_writer","compliment_photos"))

df_usercomp = df_usercomp.filter((F.col('fans') > 100) | (F.col('review_count') > 500) | (F.col('total_compliments') > 1000))

df_usercomp = df_usercomp.filter(df_usercomp.average_stars > 4.0)

df_usercomp = df_usercomp.select(df_usercomp.business_name,df_usercomp.review_count,df_usercomp.categories,df_usercomp.city,df_usercomp.state)

df_querysorted3 = df_usercomp.sort(df_usercomp.review_count.desc())

df_querysorted3 = df_querysorted3.dropDuplicates()

df_querysorted3 = df_querysorted3.limit(100000)

df_querysorted3.head(5)

Out[123]: [Row(business_name='Bacchanal Fine Wine & Spirits', review_count='2009.0', categories='Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife', city='New Orleans', state='LA'),
 Row(business_name='Bacchanal Fine Wine & Spirits', review_count='783.0', categories='Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife', city='New Orleans', state='LA'),
 Row(business_name='Bacchanal Fine Wine & Spirits', review_count='1284.0', categories='Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife', city='New Orleans', state='LA'),
 Row(business_name='Brew Brothers', review_count='1560.0', categories='Bars, Nightlife, American (Traditional), Music Venues, Food, Arts & Entertainment, Restaurants, Breweries', city='Reno', state='NV'),
 Row(business_name='Brew Brothers', review_count='573.0', categories='Bars, Nightlife, American (Traditional), Music Venues, Food, Arts & Entertain

In [ ]:
df_usercomp.write\
              .partitionBy("state")\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase2/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.businesses_vouched_by_elite_members USING DELTA LOCATION '/mnt/yelpdelta/usecase2/'")

Out[125]: DataFrame[]

In [ ]:
df_business_review_select = curated_data.select(['business_id','city','categories','business_name','review_count','stars'])

df_business_review_selectstars = df_business_review_select.filter(col('stars')>4.0)
cities = df_business_review_selectstars.select('city','business_name','stars','review_count','categories').distinct()

cities = cities.limit(100000)

display(cities)

city,business_name,stars,review_count,categories
New Orleans,Bacchanal Fine Wine & Spirits,5.0,214.0,"Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife"
New Orleans,Bacchanal Fine Wine & Spirits,5.0,6.0,"Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife"
New Orleans,Luke,5.0,38.0,"German, Restaurants, Seafood, Cocktail Bars, French, Brasseries, American (New), Nightlife, Bars, Cajun/Creole, Breakfast & Brunch"
Philadelphia,Jones,5.0,113.0,"American (New), Restaurants, Southern, Diners, American (Traditional)"
Philadelphia,Dirty Franks,5.0,137.0,"Arts & Entertainment, Bars, Jazz & Blues, Dive Bars, Nightlife"
Philadelphia,Penang,5.0,37.0,"Malaysian, Asian Fusion, Cambodian, Chinese, Restaurants, Thai"
New Orleans,Three Muses,5.0,908.0,"Nightlife, Jazz & Blues, American (New), Music Venues, Restaurants, Gastropubs, Arts & Entertainment"
New Orleans,The Praline Connection,5.0,327.0,"Seafood, Cajun/Creole, Candy Stores, Coffee & Tea, Food, Restaurants, Soul Food, Specialty Food"
New Orleans,Bacchanal Fine Wine & Spirits,5.0,51.0,"Jazz & Blues, Bars, Arts & Entertainment, Beer, Wine & Spirits, Wine Bars, Food, Nightlife"
New Orleans,Luke,5.0,1.0,"German, Restaurants, Seafood, Cocktail Bars, French, Brasseries, American (New), Nightlife, Bars, Cajun/Creole, Breakfast & Brunch"


In [ ]:
cities.write\
              .partitionBy("city")\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase3/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.cities_4_rating USING DELTA LOCATION '/mnt/yelpdelta/usecase3/'")

Out[128]: DataFrame[]

In [ ]:
# usecase2

df6 = curated_data.withColumn("fans",curated_data.fans.cast('int'))

df6 = df6.groupBy('user_id','name','fans','review_count').sum('fans')
df7 = df6.sort(col("fans").desc(),col("review_count").desc())
df7 = df7.na.drop(how="any")

df7 = df7.select("name","user_id","fans","review_count")

df7=df7.limit(100000)

display(df7)

name,user_id,fans,review_count
Daniel,j14WgRoU_-2ZE1aw1dXrJg,3138,4333.0
Peter,ITa3vh5ERI90G_WP4SmGUQ,2388,3193.0
Brittany,AHRrG3T1gJpHvtpZ-K0G_g,2086,2492.0
Farrah,3zxy3LVBV3ttxoYbY4rQ8A,2073,2073.0
Stephanie,wEE-YMx5pmSuagLtNxMPKA,1761,2000.0
Niki,WwulXySQN8t2hwqH_yWurA,1746,1512.0
Kimi,58yXn5Y4409kc9q88YwU6w,1620,1761.0
Jonathan,pnTiEaqM4slogpY97n9Kvg,1320,1251.0
Michael,vHc-UrI9yfL_pnnc6nJtyQ,1251,1135.0
Michael,U4INQZOPSUaj8hMjLlZ3KA,1238,5061.0


In [ ]:
df7.write\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase4/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.users_rating_stats USING DELTA LOCATION '/mnt/yelpdelta/usecase4/'")

Out[131]: DataFrame[]

In [ ]:
# usecase3

business_df =  curated_data.filter(length(col("state"))==2)

dff = business_df.groupBy("state","is_open").agg(count("state")).sort(col("count(state)").desc())
dff = dff.withColumnRenamed("count(state)","statewise_count")
df_f = dff.filter(col("is_open")==1)
display(df_f)

state,is_open,statewise_count
LA,1.0,112168
PA,1.0,92880
TN,1.0,37055
MO,1.0,35573
IN,1.0,30721
FL,1.0,29826
NV,1.0,24365
AZ,1.0,17105
CA,1.0,13910
NJ,1.0,3326


In [ ]:
df_f.write\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase5/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.statewise_open_businesses USING DELTA LOCATION '/mnt/yelpdelta/usecase5/'")

Out[134]: DataFrame[]

In [ ]:


users_df = curated_data.sort(col("funny").desc(),col("funny").desc(),col("compliment_funny").desc(),col("useful").desc())

df_2 = users_df.sort(col("fans").desc(),col("funny").desc(),col("compliment_funny").desc(),col("useful").desc())

df_4 = df_2.select(["compliment_funny","fans","name","useful"]).distinct()

df_4 = df_4.sort(col("fans").desc(),col("funny").desc(),col("compliment_funny").desc(),col("useful").desc()).limit(5)

In [ ]:
display(df_4)

compliment_funny,fans,name,useful
870.0,991.0,Steph,27139.0
99.0,99.0,Quisha,2018.0
71.0,99.0,Sam,2429.0
209.0,99.0,Angel,1347.0
32.0,99.0,Debbie,1285.0


In [ ]:
df_4.write\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/yelpdelta/usecase6/")

In [ ]:
spark.sql("CREATE TABLE if not exists dsddb.top_5_useful_users USING DELTA LOCATION '/mnt/yelpdelta/usecase6/'")

Out[191]: DataFrame[]